<h3>Python DB-API</h3>

In [ ]:
# https://www.postgresqltutorial.com/postgresql-python/connect/

PG_HOST = 'localhost'
PG_PORT = 5432
PG_DATABASE = 'ntu'
PG_USER = 'postgres'
PG_PASS = '1234'

import psycopg2
    
def connect():
    return psycopg2.connect(host=PG_HOST, database=PG_DATABASE, user=PG_USER, password=PG_PASS)

In [1]:
# use this cell instead of the previous one if you want use sqlite instead of postgresql

import sqlite3

def connect():
    return sqlite3.connect('../some.db')

<h3>SQL</h3>

In [4]:
def execute_sql(conn, query, parameters=None):
    conn = connect()
    cursor = conn.cursor()
    if parameters is None:
        cursor.execute(query)
    else:
        cursor.execute(query, parameters)

    if query.lower().lstrip().startswith('select'):
        result = cursor.fetchall()
    else:
        result = None
    conn.commit()
    return result

<h3>SELECT & NULL</h3>

In [5]:
c = connect();

print(execute_sql(c, 'SELECT 1'))
print(execute_sql(c, 'SELECT 1+3'))

print(execute_sql(c, 'SELECT NULL = 1'))
print(execute_sql(c, 'SELECT NULL != 1'))
print(execute_sql(c, 'SELECT NULL = NULL'))
print(execute_sql(c, 'SELECT NULL != NULL'))

print(execute_sql(c, 'SELECT NULL IS NULL'))
print(execute_sql(c, 'SELECT NULL IS NOT NULL'))

c.close();

[(1,)]
[(4,)]
[(None,)]
[(None,)]
[(None,)]
[(None,)]
[(1,)]
[(0,)]


<h3>Database Initialization</h3>

In [7]:
def create_test_data(c):
    
    execute_sql(c, """
        CREATE TABLE Bachelor(id INTEGER PRIMARY KEY, name TEXT, course TEXT);
    """)

    execute_sql(c, """
        INSERT INTO Bachelor(id, name, course) VALUES
        (1, 'Болванщик', 'СУБД'),
        (2, 'Заяц', 'C++'),
        (3, 'Соня', 'Алгоритмы');
    """)

    execute_sql(c, """
        CREATE TABLE Master(id INTEGER PRIMARY KEY, name TEXT, course TEXT);
    """)

    execute_sql(c, """
        INSERT INTO Master(id, name, course) VALUES
        (10, 'Алиса', 'СУБД'),
        (20, 'Алиса', 'C++');
    """)

    
with connect() as c:
    create_test_data(c)

<h3>SELECT</h3>

In [8]:
with connect() as c:
    r = execute_sql(c, "SELECT * FROM Bachelor")

print(r)

[(1, 'Болванщик', 'СУБД'), (2, 'Заяц', 'C++'), (3, 'Соня', 'Алгоритмы')]


In [9]:
with connect() as c:
    r = execute_sql(c, "SELECT * FROM Master")

print(r)

[(10, 'Алиса', 'СУБД'), (20, 'Алиса', 'C++')]


In [10]:
with connect() as c:
    r = execute_sql(c, "SELECT * FROM Bachelor, Master")

print(r)

[(1, 'Болванщик', 'СУБД', 10, 'Алиса', 'СУБД'), (1, 'Болванщик', 'СУБД', 20, 'Алиса', 'C++'), (2, 'Заяц', 'C++', 10, 'Алиса', 'СУБД'), (2, 'Заяц', 'C++', 20, 'Алиса', 'C++'), (3, 'Соня', 'Алгоритмы', 10, 'Алиса', 'СУБД'), (3, 'Соня', 'Алгоритмы', 20, 'Алиса', 'C++')]


<h3>JOINS</h3>

In [11]:
# INNER JOIN

sql_v1 = """
    SELECT *
       FROM Bachelor, Master
       WHERE Master.name = 'Алиса'
          AND Bachelor.course = Master.course;
"""

sql_v2 = """
    SELECT *
       FROM Bachelor
          JOIN Master ON Bachelor.course = Master.course
       WHERE Master.name = 'Алиса';
"""

with connect() as c:
    r = execute_sql(c, sql_v2)

print(r)

[(1, 'Болванщик', 'СУБД', 10, 'Алиса', 'СУБД'), (2, 'Заяц', 'C++', 20, 'Алиса', 'C++')]


In [12]:
# LEFT JOIN

sql = """
    SELECT *
       FROM Bachelor
          LEFT JOIN Master ON Bachelor.course = Master.course;
"""

with connect() as c:
    r = execute_sql(c, sql)
    
print(r)

[(1, 'Болванщик', 'СУБД', 10, 'Алиса', 'СУБД'), (2, 'Заяц', 'C++', 20, 'Алиса', 'C++'), (3, 'Соня', 'Алгоритмы', None, None, None)]


In [13]:
# RIGHT JOIN

sql = """
    SELECT *
       FROM Master
          RIGHT JOIN Bachelor ON Bachelor.course = Master.course;
"""

with connect() as c:
    r = execute_sql(c, sql)
    
print(r)

[(10, 'Алиса', 'СУБД', 1, 'Болванщик', 'СУБД'), (20, 'Алиса', 'C++', 2, 'Заяц', 'C++'), (None, None, None, 3, 'Соня', 'Алгоритмы')]


In [16]:
type(r), type(r[0]), type(r[0][0])

(list, tuple, int)